сверточный автоэнкодер

In [1]:
import numpy as np
import keras
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2 as cv


#генерация изображений с помощью ImageDataGenerator библиотеки Keras
def loadData(h, w):
  train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
  train_generator=train_datagen.flow_from_directory('/content/sample_data/data_pic',
                                                  target_size=(w,h),
                                                  color_mode='grayscale',
                                                  batch_size=32,
                                                  class_mode='categorical', shuffle=True)
  return train_generator[0][0]

In [2]:
loadData(32, 32)

Found 16 images belonging to 2 classes.


array([[[[-0.4980392 ],
         [-0.38039213],
         [-0.27058822],
         ...,
         [-0.94509804],
         [-0.92156863],
         [-0.90588236]],

        [[-0.49019605],
         [-0.3098039 ],
         [-0.24705881],
         ...,
         [-0.92941177],
         [-0.92156863],
         [-0.9137255 ]],

        [[-0.4352941 ],
         [-0.27058822],
         [-0.27058822],
         ...,
         [-0.9137255 ],
         [-0.9529412 ],
         [-0.88235295]],

        ...,

        [[-0.9372549 ],
         [-0.92941177],
         [-0.9137255 ],
         ...,
         [-0.15294117],
         [ 0.0196079 ],
         [ 0.04313731]],

        [[-0.94509804],
         [-0.9137255 ],
         [-0.9137255 ],
         ...,
         [-0.11372548],
         [ 0.06666672],
         [ 0.11372554]],

        [[-0.9372549 ],
         [-0.92941177],
         [-0.9529412 ],
         ...,
         [-0.6313726 ],
         [-0.0745098 ],
         [ 0.082353  ]]],


       [[[-0.17647058],


In [8]:
from sklearn.base import BaseEstimator, ClassifierMixin
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,  UpSampling2D
from keras import backend as K
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

batch_size = 2
epochs = 50

h, w = 32, 32
input_shape = (h, w, 1)
y_train = x_train = loadData(h, w)

class KerasGridSearchWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, p1=1, p2=4, p3=16, batch_size=2, epochs=50):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.batch_size = batch_size
        self.epochs = epochs

    def fit(self, X, y):
        input_shape = X.shape[1:]
        self.model = Sequential()
        self.model.add(Conv2D(self.p1, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Conv2D(self.p2, (3, 3), activation='relu', padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Conv2D(self.p3, (3, 3), activation='relu', padding='same'))
        self.model.add(UpSampling2D(size=(2, 2)))
        self.model.add(Conv2D(self.p2, (3, 3), activation='relu', padding='same'))
        self.model.add(UpSampling2D(size=(2, 2)))
        self.model.add(Conv2D(self.p1, (3, 3), activation='linear', padding='same'))
        self.model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
        self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs)
        return self

    def score(self, X, y):
        return self.model.evaluate(X, y, verbose=0)[1]


y_train = x_train = loadData(h, w)

param_grid = {
    'p1': [1, 4],
    'p2': [4, 8],
    'p3': [16, 24]
}

model = GridSearchCV(estimator=KerasGridSearchWrapper(), param_grid=param_grid, cv=3)
model.fit(x_train, y_train)

print(model.best_params_)

Found 16 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Epoch 1/50
5/5 [==============================] - 1s 7ms/step - loss: 0.2502 - accuracy: 0.0000e+00
Epoch 2/50
5/5 [==============================] - 0s 6ms/step - loss: 0.2152 - accuracy: 0.0000e+00
Epoch 3/50
5/5 [==============================] - 0s 6ms/step - loss: 0.2045 - accuracy: 0.0000e+00
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1944 - accuracy: 0.0000e+00
Epoch 5/50
5/5 [==============================] - 0s 5ms/step - loss: 0.1877 - accuracy: 0.0000e+00
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1793 - accuracy: 0.0000e+00
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1716 - accuracy: 0.0000e+00
Epoch 8/50
5/5 [==============================] - 0s 8ms/step - loss: 0.1645 - accuracy: 0.0000e+00
Epoch 9/50
5/5 [==============================] - 0s 5ms/step - loss: 0.1584 - accuracy: 0.0000e+00
Epoch 10/50
5/5 [===

Epoch 1/50
6/6 [==============================] - 1s 5ms/step - loss: 0.2392 - accuracy: 0.0000e+00
Epoch 2/50
6/6 [==============================] - 0s 5ms/step - loss: 0.2213 - accuracy: 0.0000e+00
Epoch 3/50
6/6 [==============================] - 0s 5ms/step - loss: 0.2038 - accuracy: 0.0000e+00
Epoch 4/50
6/6 [==============================] - 0s 5ms/step - loss: 0.1818 - accuracy: 0.0000e+00
Epoch 5/50
6/6 [==============================] - 0s 5ms/step - loss: 0.1636 - accuracy: 0.0000e+00
Epoch 6/50
6/6 [==============================] - 0s 6ms/step - loss: 0.1450 - accuracy: 0.0000e+00
Epoch 7/50
6/6 [==============================] - 0s 5ms/step - loss: 0.1258 - accuracy: 0.0000e+00
Epoch 8/50
6/6 [==============================] - 0s 5ms/step - loss: 0.1151 - accuracy: 0.0000e+00
Epoch 9/50
6/6 [==============================] - 0s 5ms/step - loss: 0.1116 - accuracy: 0.0000e+00
Epoch 10/50
6/6 [==============================] - 0s 5ms/step - loss: 0.1077 - accuracy: 0.0000e+00

Epoch 1/50
5/5 [==============================] - 1s 6ms/step - loss: 0.2208 - accuracy: 0.0000e+00
Epoch 2/50
5/5 [==============================] - 0s 5ms/step - loss: 0.2085 - accuracy: 0.0000e+00
Epoch 3/50
5/5 [==============================] - 0s 5ms/step - loss: 0.1980 - accuracy: 0.0000e+00
Epoch 4/50
5/5 [==============================] - 0s 5ms/step - loss: 0.1829 - accuracy: 0.0000e+00
Epoch 5/50
5/5 [==============================] - 0s 5ms/step - loss: 0.1657 - accuracy: 0.0000e+00
Epoch 6/50
5/5 [==============================] - 0s 5ms/step - loss: 0.1454 - accuracy: 0.0000e+00
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1356 - accuracy: 0.0000e+00
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1297 - accuracy: 0.0000e+00
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1259 - accuracy: 0.0000e+00
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1248 - accuracy: 0.0000e+00